In [1]:
## Import Dependencies
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import seaborn as sns
import geopandas as gpd
import plotly.express as px

## Import our functions for converting countries to continents for analysis
import pycountry
import pycountry_convert as pc
from pycountry_convert import country_name_to_country_alpha2
from pycountry_convert import country_alpha2_to_continent_code

## Import Our Datasets and Reformat

In [2]:
## Pulling data from csvs and storing as dataframes and Cleaning our dataframes

## Import Life Expectancy per Country per Year dataset
life_expect_csv = os.path.join(os.getcwd(), "datasets", "lifeExpectancyWorld.csv")
life_expect_df = pd.read_csv(life_expect_csv)

## Changing column names
life_expect_df = life_expect_df.drop(columns = {"Code"})
life_expect_df = life_expect_df.rename(columns = {'Entity' : 'Location'})
life_expect_df = life_expect_df.melt(['Location'], var_name = 'Period', value_name = 'Life Expectancy')
life_expect_df['Period'] = life_expect_df['Period'].astype(int)
life_expect_df = life_expect_df.rename(columns = {"Entity" : "Location",
                                                 "Year" : "Period"
                                                 })

In [3]:
## Import Pharmacists per 10,000 dataset
pharm_csv = os.path.join(os.getcwd(), "datasets", "pharmacists.csv")
pharm_df = pd.read_csv(pharm_csv)

## Changing column names
pharm_df = pharm_df.rename(columns = {"First Tooltip" : "Pharmicists Per 10,000"}).drop(columns =["Indicator"])

In [4]:
## Import Medical Doctors per 10,000 dataset
med_doctors_csv = os.path.join(os.getcwd(), "datasets", "medicalDoctors.csv")
med_doctors_df = pd.read_csv(med_doctors_csv)

## Changing column names
med_doctors_df = med_doctors_df.rename(columns = {"First Tooltip" : "Medical Doctors Per 10,000"}).drop(columns =["Indicator"])

In [5]:
## Import % of Population with Access to Basic Handwashing Facilities at Home dataset
hand_wash_csv = os.path.join(os.getcwd(), "datasets", "handWashing.csv")
hand_wash_df = pd.read_csv(hand_wash_csv)

## Changing column names
hand_wash_df = hand_wash_df.rename(columns = {"First Tooltip" : "Population with basic handwashing facilities at home (%)"}).drop(columns =["Indicator"])
hand_wash_df = hand_wash_df[hand_wash_df["Dim1"].str.contains("Urban")==False]
hand_wash_df = hand_wash_df[hand_wash_df["Dim1"].str.contains("Rural")==False]
hand_wash_df = hand_wash_df.drop(columns = {"Dim1"})

In [6]:
## Import % of Population Using at Least Basic Drinking Water Services dataset
drink_water_csv = os.path.join(os.getcwd(), "datasets", "drinkingWaterServices.csv")
drink_water_df = pd.read_csv(drink_water_csv)

##Changing column names
drink_water_df = drink_water_df.rename(columns = {"First Tooltip" : "Population using at least basic drinking-water services (%)"}).drop(columns =["Indicator"])

In [7]:
## Import % of Population Using at least Basic Sanitation Services dataset
sanitize_csv= os.path.join(os.getcwd(), "datasets", "sanitationServices.csv")
sanitize_df = pd.read_csv(sanitize_csv)

##Changing column names
sanitize_df = sanitize_df.rename(columns = {"First Tooltip" : "Population using at least basic sanitization services (%)"}).drop(columns =["Indicator"])
sanitize_df = sanitize_df[sanitize_df["Dim1"].str.contains("Urban")==False]
sanitize_df = sanitize_df[sanitize_df["Dim1"].str.contains("Rural")==False]
sanitize_df = sanitize_df.drop(columns = {"Dim1"})

## Start the Merging Process

In [8]:
## Perform outer merges of our dataframes, merge on "Location" and "Period"

## Merge the medical doctors and pharmacists dataframes
merge_df1 = pd.merge(pharm_df, med_doctors_df, on = ["Location", "Period"], how = "outer")

In [9]:
## Merge the previous dataframe with the life expectancy dataframe
merge_df2 = pd.merge(merge_df1, life_expect_df, on = ["Location", "Period"], how = "outer")

In [10]:
## Merge the previous dataframe with the hand washing datframe
merge_df3 = pd.merge(merge_df2, hand_wash_df, on = ["Location", "Period"], how = "outer")

In [11]:
## Merge the previous dataframe with the drinking water dataframe
merge_df4 = pd.merge(merge_df3, drink_water_df, on = ["Location", "Period"], how = "outer")

In [12]:
## Merge the previous dataframe with the sanitation services dataframe (This is the last merge)
final_merge_df = pd.merge(merge_df4, sanitize_df, on = ["Location", "Period"], how = "outer")

In [13]:
## Let's look at the final product
final_merge_df

,Location,Period,"Pharmicists Per 10,000","Medical Doctors Per 10,000",Life Expectancy,Population with basic handwashing facilities at home (%),Population using at least basic drinking-water services (%),Population using at least basic sanitization services (%)
0,Afghanistan,2016,0.47,2.78,63.763,37.67,54.84,42.05
1,Afghanistan,2015,0.50,2.85,63.377,37.59,52.39,40.71
2,Afghanistan,2014,0.51,2.98,62.966,37.52,49.96,39.37
3,Afghanistan,2012,0.27,2.41,62.054,37.37,45.19,36.75
4,Afghanistan,2011,0.28,2.52,61.553,37.30,42.84,35.46
...,...,...,...,...,...,...,...,...
17421,Viet Nam,2004,NaN,NaN,NaN,NaN,79.45,59.79
17422,Viet Nam,2003,NaN,NaN,NaN,NaN,78.47,57.95
17423,Viet Nam,2000,NaN,NaN,NaN,NaN,75.54,52.37
17424,Republic of Korea,2001,NaN,NaN,NaN,NaN,NaN,100.00


In [14]:
## Some country names don't work well with our contry to continent conversion function (Côte d’Ivoire, Sudan (until 2011), etc), so let's change a few
for index, row in final_merge_df.iterrows():
    country = row['Location']
    if country == "Bolivia (Plurinational State of)":
        change_country = "Bolivia"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Côte d’Ivoire":
        change_country = "Ivory Coast"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Iran (Islamic Republic of)":
        change_country = "Iran"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Republic of Korea":
        change_country = "South Korea"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Sudan (until 2011)":
        change_country = "Sudan"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Timor-Leste":
        change_country = "East Timor"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Micronesia (Federated States of)":
        change_country = "Micronesia"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "The former Yugoslav Republic of Macedonia":
        change_country = "North Macedonia"
        final_merge_df.loc[ index, "Location"] = change_country
    if country == "Venezuela (Bolivarian Republic of)":
        change_country = "Venezuela"
        final_merge_df.loc[ index, "Location"] = change_country

In [15]:
## let's see how well they cleaned up
final_merge_df.loc[final_merge_df["Location"] == "Venezuela"]

,Location,Period,"Pharmicists Per 10,000","Medical Doctors Per 10,000",Life Expectancy,Population with basic handwashing facilities at home (%),Population using at least basic drinking-water services (%),Population using at least basic sanitization services (%)
2613,Venezuela,2001,NaN,19.48,NaN,NaN,96.60,NaN
2860,Venezuela,1950,NaN,NaN,54.273,NaN,NaN,NaN
3103,Venezuela,1951,NaN,NaN,54.574,NaN,NaN,NaN
3346,Venezuela,1952,NaN,NaN,55.173,NaN,NaN,NaN
3589,Venezuela,1953,NaN,NaN,55.769,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
17416,Venezuela,2005,NaN,NaN,NaN,NaN,96.38,92.54
17417,Venezuela,2004,NaN,NaN,NaN,NaN,96.43,NaN
17418,Venezuela,2003,NaN,NaN,NaN,NaN,96.49,NaN
17419,Venezuela,2002,NaN,NaN,NaN,NaN,96.54,NaN


In [ ]:
## Now we have the issue that some locations in our dataset don't convert at all into countries (such as certain territories, islands, or broad areas)
## so we're going to skip them and leave them out of our groupby country analysis

#Put info into Continent Row based on Location
index_list = []
problem_countries = []
for index, row in final_merge_df.iterrows():
    
    # get continent type from df
    country = row['Location']
    try:
        # print(country)
        c_to_a = country_name_to_country_alpha2(country)
        final_merge_df.loc[index, 'AFF_ISO']= c_to_a
        # print(c_to_a)    
        c_to_c = country_alpha2_to_continent_code(c_to_a)   
        final_merge_df.loc[index, 'Continent']= c_to_c
        
    ## If the country doesn't convert, leave it blank
    except KeyError:
        ## If we just wanted to drop our problem locations, we use the following:
        ## row_to_drop = index
        ## index_list.append(row_to_drop)
        final_merge_df.loc[index, 'Continent']= np.NaN
        ## Also, let's collect those problem countries in a list for future reference
        problem_countries.append(country)
        
## Now to drop the problem countries, we could use this: 
## final_merge_df = final_merge_df.drop(index_list)

In [ ]:
## Let's see how many problem locations we have
len(problem_countries)

In [ ]:
## Let's make sure this worked
final_merge_df.loc[final_merge_df["Location"] == "East Timor"].head(5)

In [ ]:
## Now let's make sure we can groupby with no errors
final_merge_df.groupby(["Continent"]).describe()

In [ ]:
## Nothing seems to be out of the ordinary with our desciptions so let's move onto the analysis!

## Begin Analysis!

In [ ]:
## First, let's get a rolling average life expectancy for our continents to get a understanding of how life expectancy is changing

## Create series groupby of average life expectancy per year per continent
avg_lifeExp_per_country_per_year = final_merge_df.groupby(["Continent", "Period"])["Life Expectancy"].mean()
avg_lifeExp_per_country_per_year

In [ ]:
## Turn that series into a dataframe and reset the index
group_df = avg_lifeExp_per_country_per_year.to_frame().reset_index()
group_df

In [ ]:
## Pull individual continent data into their own dataframes for graphing
af = group_df.loc[group_df["Continent"] == "AF"]
asia = group_df.loc[group_df["Continent"] == "AS"]
eu = group_df.loc[group_df["Continent"] == "EU"]
na = group_df.loc[group_df["Continent"] == "NA"]
oc = group_df.loc[group_df["Continent"] == "OC"]
sa = group_df.loc[group_df["Continent"] == "SA"]

In [ ]:
## PLot the data
plt.figure(figsize=(10,7))
plt.plot( af["Period"], af["Life Expectancy"], label = "Africa")
plt.plot( asia["Period"], asia["Life Expectancy"], label = "Asia")
plt.plot( eu["Period"], eu["Life Expectancy"], label = "Europe")
plt.plot( na["Period"], na["Life Expectancy"], label = "North America")
plt.plot( oc["Period"], oc["Life Expectancy"], label = "Oceania")
plt.plot( sa["Period"], sa["Life Expectancy"], label = "South America")
plt.legend(loc = "lower right")
plt.title("Average Life Expectancy Per Year by Continent")
plt.xlabel("Year")
plt.ylabel("Average Life Expectancy (Years)")
plt.savefig("graphs/Average Life Expectancy Per Year by Continent.png")
plt.show()

#### Bar Graphs

In [ ]:
## Clearly, life expectancy is increasing in general
## Let's now look at some basic bar graphs to see other trends

plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Life Expectancy"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Life Expectancy")
plt.title("Average Life Expectancy per Continent")

In [ ]:
## Avg # of Pharmicists per 10,000 per Continent
plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Pharmicists Per 10,000"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Number of Pharmicists Per 10,000")
plt.title("Average Number of Pharmicists Per 10,000 per Continent")

In [ ]:
## Avg % of Pop with basic handwashing facilities at home per Continent
plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Medical Doctors Per 10,000"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Number of Medical Doctors Per 10,000")
plt.title("Average Number of Medical Doctors Per 10,000 per Continent")

In [ ]:
## Avg % of Pop with basic handwashing facilities at home per Continent
plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Population with basic handwashing facilities at home (%)"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Population with basic handwashing facilities at home (%)")
plt.title("Average Population with basic handwashing facilities at home (%) per Continent")
plt.show()

In [ ]:
## Average % of pop using at least basic drinking water services by country
plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Population using at least basic drinking-water services (%)"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Population using at least basic drinking-water services (%)")
plt.title("Average Population using at least basic drinking-water services (%) per Continent")
plt.show()

In [ ]:
## Average % of pop using at least basic sanitation services by country
plt.figure(figsize = (10, 7))
final_merge_df.groupby(["Continent"])["Population using at least basic sanitization services (%)"].mean().plot(kind = 'bar', color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown'])
plt.ylabel("Average Population using at least basic sanitization services (%)")
plt.title("Average Population using at least basic sanitization services (%) per Continent")
plt.show()

#### Regression

In [ ]:
## Now let's run some regressions against life expectancy to see if any of these variables correlate with it

plt.figure(figsize = (10, 7))
x_values = final_merge_df["Life Expectancy"]
y_values = final_merge_df["Population using at least basic drinking-water services (%)"]
mask = ~np.isnan(x_values) & ~np.isnan(y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values[mask], y_values[mask])
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlim(30,90)
plt.ylim(0,103)
plt.annotate(line_eq,(40,80),fontsize=15,color="red")
plt.title("Population using at least basic drinking-water services (%) vs Life Expectancy")
plt.xlabel("Life Expectancy")
plt.ylabel("Population using at least basic drinking-water services (%)")
print(f"The r-squared is: {rvalue**2}")
plt.savefig("graphs/Life Expectancy vs Population using at least basic drinking-water services (%).png")
plt.show()

In [ ]:
plt.figure(figsize = (10, 7))
x_values = final_merge_df["Life Expectancy"]
y_values = final_merge_df["Population with basic handwashing facilities at home (%)"]
mask = ~np.isnan(x_values) & ~np.isnan(y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values[mask], y_values[mask])
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlim(42,85)
plt.ylim(-5,103)
plt.annotate(line_eq,(45,80),fontsize=15,color="red")
plt.title("Population with basic handwashing facilities at home (%) vs Life Expectancy")
plt.xlabel("Life Expectancy")
plt.ylabel("Population with basic handwashing facilities at home (%)")
print(f"The r-squared is: {rvalue**2}")
plt.savefig("graphs/Life Expectancy vs Population with basic handwashing facilities at home (%).png")
plt.show()

In [ ]:
plt.figure(figsize = (10, 7))
x_values = final_merge_df["Life Expectancy"]
y_values = final_merge_df["Population using at least basic sanitization services (%)"]
mask = ~np.isnan(x_values) & ~np.isnan(y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values[mask], y_values[mask])
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlim(36,90)
plt.ylim(-5,103)
plt.annotate(line_eq,(45,90),fontsize=15,color="red")
plt.title("Population using at least basic sanitization services (%) vs Life Expectancy")
plt.xlabel("Life Expectancy")
plt.ylabel("Population using at least basic sanitization services (%)")
print(f"The r-squared is: {rvalue**2}")
plt.savefig("graphs/Life Expectancy vs Population using at least basic sanitization services (%).png")
plt.show()

In [ ]:
plt.figure(figsize = (10, 7))
x_values = final_merge_df["Life Expectancy"]
y_values = final_merge_df["Pharmicists Per 10,000"]
mask = ~np.isnan(x_values) & ~np.isnan(y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values[mask], y_values[mask])
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlim(36,86)
plt.ylim(-1,28)
plt.annotate(line_eq,(50,15),fontsize=15,color="red")
plt.title("Pharmicists Per 10,000 vs Life Expectancy ")
plt.xlabel("Life Expectancy")
plt.ylabel("Pharmicists Per 10,000")
print(f"The r-squared is: {rvalue**2}")
plt.savefig("graphs/Life Expectancy vs Pharmicists Per 10,000.png")
plt.show()

In [ ]:
plt.figure(figsize = (10, 7))
x_values = final_merge_df["Life Expectancy"]
y_values = final_merge_df["Medical Doctors Per 10,000"]
mask = ~np.isnan(x_values) & ~np.isnan(y_values)
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values[mask], y_values[mask])
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlim(36,86)
plt.ylim(-4,89)
plt.annotate(line_eq,(50,15),fontsize=15,color="red")
plt.title("Medical Doctors Per 10,000 vs Life Expectancy")
plt.xlabel("Life Expectancy")
plt.ylabel("Medical Doctors Per 10,000")
print(f"The r-squared is: {rvalue**2}")
plt.savefig("graphs/Life Expectancy vs Medical Doctors Per 10,000.png")
plt.show()

#### Correlation Matrix

In [ ]:
## Now let's take a look at how each of our variables interact with each other via correlation matrix
corr_df = final_merge_df.corr()
heatmap_df = corr_df.drop(['Period']).drop(['Period'],axis=1)
heatmap_df

In [ ]:
plt.figure(figsize = (10, 7))
plt.title("Correlation Matrix")
sns.heatmap(heatmap_df, annot=True)
plt.savefig("graphs/Correlation Matrix.png")
plt.show()

#### Mapping Our Data

In [ ]:
## Let's take a look graphically at how each country compares in terms of average life expectancy

## First, import a shape file for the world so we can plot this stuff
fp = "resources/World_Countries__Generalized_.shp"
map_df = gpd.read_file(fp)

In [ ]:
map_df

In [ ]:
## Filter out AFF ISO and average Life Epectancy from the original df, pull into another dataframe, and reformat for merging
iso_df = final_merge_df.filter(["AFF_ISO", "Life Expectancy"], axis =1).set_index("AFF_ISO")
iso_df = iso_df.groupby(["AFF_ISO"])["Life Expectancy"].mean().to_frame()
iso_df = iso_df.rename(columns = {"Life Expectancy" : "avg_life"})

In [ ]:
## Now merge on AFF ISO to get avg life expectancy into a dataframe
map_merge_df = pd.merge(map_df, iso_df, on ="AFF_ISO")
map_merge_df

In [ ]:
## Convert the merged map dataframe into a geodataframe
gdf = gpd.GeoDataFrame(map_merge_df)

In [ ]:
## Now plot!
gdf.plot(column = "avg_life", 
         figsize =(18,11), 
         legend = True, 
         cmap='OrRd', 
         legend_kwds={'label': "Average Life Expectancy", 
                      'orientation': "horizontal"}).set_title("Average Life Expectancy Per Country")
plt.savefig("graphs/Average Life Expectancy by Continent.png")